첫 샘플의 베스트

"reindeer mistletoe elf gingerbread family advent scrooge chimney fireplace ornament"

## imports

In [3]:
import gc ## for memory
import os
import copy
from math import exp
from collections import Counter
from typing import List, Optional, Union

import numpy as np
import pandas as pd
import transformers
import torch

from kaggle_evaluate import PerplexityCalculator

In [4]:
df_sample = pd.read_csv("sample_submission.csv")

## setting

In [5]:
evaluatr = PerplexityCalculator("google/gemma-2-9b")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [9]:
greedy_starter = pd.read_csv("greedy2.csv", index_col = 0)

In [17]:
evaluatr.get_perplexity(greedy.text.to_list(), batch_size=1024)

[467.98558773246197,
 840.8178087002444,
 967.7753655846766,
 762.5886393094895,
 302.15420510895507,
 120.65904401308495]

`-` beam size : 

In [ ]:
all_tokens = greedy_starter.text.str.split().to_list()
beam_size = [5, 10, 10, 15, 25, 30]
best_word = []

for i, tokens in enumerate(all_tokens) :
    token_start = tokens[0]
    width = beam_size[i]
    token_list = [copy.deepcopy(tokens[1:]) for _ in range(width)]
    beam_set = []
    
    for j in range(len(tokens)-1) :
        word_list = []
        
        if beam_set == [] :            
            for t in token_list[0] :
                word_list.append(" ".join([token_start] + [t]))
            
            perplexities = np.array(evaluatr.get_perplexity(word_list, batch_size = 1024))
            min_indexs = perplexities.argsort()[:width]
            token_len = len(token_list[0])
            beam_set = [[token_start, token_list[min_index//token_len][min_index%token_len]] for min_index in min_indexs]
        
            for l in range(width) :
                del token_list[l][min_indexs[l]%token_len]
            
        else :
            for m, beam in enumerate(beam_set) :
                for t in token_list[m] :
                    word_list.append(" ".join(beam + [t]))
                
            perplexities = np.array(evaluatr.get_perplexity(word_list, batch_size = 1024))
            min_indexs = perplexities.argsort()[:width]
            token_len = len(token_list[0])
            tmp = copy.deepcopy(beam_set)
            beam_set = [tmp[min_index//token_len] + [token_list[min_index//token_len][min_index%token_len]] for min_index in min_indexs]
        
            tmp = copy.deepcopy(token_list)
            
            for l, min_index in enumerate(min_indexs) :
                token_list[l] = copy.deepcopy(tmp[min_index//token_len])
                del token_list[l][min_index%token_len]
                
        
    best_word.append(beam_set[perplexities.argmax()])

In [ ]:
df_submission = pd.DataFrame({"id":[0,1,2,3,4,5], "text":[" ".join(w) for w in best_word]})
df_submission.to_csv("greedy2.csv")

os.system("kaggle competitions submit -c santa-2024 -f greedy2.csv -m .")